# Individual plant / tree segmentation with lidR

[lidR](https://github.com/Jean-Romain/lidR/wiki) is a R package created by Jean-Romain. It is built natively in R, and allows users to select from multiple segmentation techniques (e.g. watershed, 

Example scripts provide below were written by Andrew Sanchez-Meador (Northen Arizona University), adapted for this example by Tyson L. Swetnam (University of Arizona)

## Dependencies

```
conda install -c r r-devtools
```

## Install Libraries & Dependencies

In [ ]:
packageVersion('IRdisplay')
packageVersion('IRkernel')
packageVersion('repr')

In [ ]:
install.packages(c("rgdal","lidR","IRdisplay"))

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


Load libraries

In [ ]:
library(lidR)
library(rgdal)
library(leaflet)

Set options

In [ ]:
lidr_options(verbose = TRUE, progress = TRUE)

## Build a project 

Set the working directory where the lidar tiles are, and where the output will be

In [38]:
getwd()

[1] "/home/tswetnam/QUBES_NEON/lessons/srer_laz"

In [39]:
setwd('/home/tswetnam/QUBES_NEON/lessons/srer_laz/')

In [ ]:
project <- catalog("/home/tswetnam/QUBES_NEON/lessons/srer_laz/classified/NEON_D14_SRER_DP1_502000_3523000_classified_point_cloud.laz")

In [ ]:
plot(project)

In [ ]:
las = readLAS("/home/tswetnam/QUBES_NEON/lessons/srer_laz/classified/NEON_D14_SRER_DP1_502000_3523000_classified_point_cloud.laz")

In [ ]:
las

## Set some global catalog options

For this dummy example, the clustering size is 500 m and the buffer is 15 m using
a single core (because this example is run on the CRAN server when the package is submitted).

In [ ]:
catalog_options(buffer = 15, multicore = 8, tiling_size = 500)

## Build the function that analyzes each cluster of the catalog.

The function's first argument is a LAS object. The internal routine takes care of this part. The other arguments can be freely choosen by the user. See the following template:

In [ ]:
tree_area = function(las)
{
  if (is.null(las))
    return(NULL)
  
  # segment trees (in this example the low point density does not enable
  # accurate segmentation of trees. This is just a proof-of-concept)
  lastrees(las, algorithm = "li2012")
  
  # Here we used the function tree_metric to compute some metrics for each tree. This
  # function is defined later in the global environment.
  m = tree_metrics(las, myMetrics(X, Y, Z, buffer, treeID))
  
  # If min buffer is 0 it means the trees were at least partly in the non-buffered area, so we
  # want to keep these trees.
  # However, the trees that are on the edge of the buffered area will be counted
  # twice. So we must remove the trees on the right side and on the top side of the buffer
  # If max buffer is <= 2 it means that the trees belong inside the area of interest, on
  # the left side or the bottom side, or both.
  m = m[minbuff == 0 & maxbuff <= 2]
  
  # Remove buffering information that is no longer useful
  m[, c("minbuff","maxbuff") := NULL]
  
  return(m)
}

This function enables users to extract, for a single tree, the position of the highest point and some information about the buffering position of the tree. The function tree_metrics takes care of mapping along each tree.

In [ ]:
myMetrics <- function(x, y, z, buff, treeID)
{
  i = which.max(z)
  xcenter = x[i]
  ycenter = y[i]
  A = area(x,y)
  dat = matrix(c(x, y), nrow=length(x), ncol=2)
  ch = chull(dat)
  coords <- dat[c(ch, ch[1]), ]  
  sp_poly <- SpatialPolygons(list(Polygons(list(Polygon(coords)), ID=treeID[i])))
  minbuff = min(buff)
  maxbuff = max(buff)
  
  return(
    list(
      x = xcenter,
      y = ycenter,
      area = A,
      crowns=list(sp_poly),
      minbuff = minbuff,
      maxbuff = maxbuff
    ))
}

Everything is now well defined, so now we can process over an entire catalog with hundreds of files (but in this example we use just one file...)

## Process the project. 

The arguments of the user-defined function must belong in a labelled list. We also pass extra arguments to the function readLAS  to load only X, Y and Z coordinates. This way we save a huge amount of memory, which can be used for the current process.

In [ ]:
output = catalog_apply(project, tree_area, select = "xyz")

## Post-process the output result 

(depending on the output computed). Here, each value of the list is a data.table, so rbindlist does the job:

In [ ]:
output = data.table::rbindlist(output)

Create a list of polygons to store the crown chulls for now

In [ ]:
list_of_SPols = output$crowns

In [ ]:
Now remove the crown informaiton from te output, to prepare to write the geojson

In [ ]:
output = output[,1:4]

In [ ]:
library(lidR)

# read file
las = readLAS("/home/tswetnam/Desktop/DiscreteLidar/ClassifiedPointCloud/NEON_D14_SRER_DP1_514000_3519000_classified_point_cloud.laz")

# normalization
las = lasnormalize(las, method = "kriging")

# compute a canopy image
chm = grid_tincanopy(las, 0.25, c(0,2,5,10,15), c(0,1) , subcircle = 0.2)
chm = as.raster(chm)
kernel = matrix(1,3,3)
chm = raster::focal(chm, w = kernel, fun = mean)
chm = raster::focal(chm, w = kernel, fun = mean)

# tree segmentation
crowns = lastrees(las, "watershed", chm, th = 0.25, extra = TRUE)

# display
tree = lasfilter(las, !is.na(treeID))
plot(tree, color = "treeID", colorPalette = pastel.colors(100), size = 2)

# More stuff
library(raster)
contour = rasterToPolygons(crowns, dissolve = TRUE)

plot(chm, col = height.colors(50))
plot(contour, add = T)